In [2]:
import numpy as np
import pandas as pd

# scale
from sklearn.preprocessing import StandardScaler

# model selectiion
from sklearn.model_selection import StratifiedKFold, cross_val_score

# rf
from sklearn.ensemble import RandomForestClassifier
# mlp
from sklearn.neural_network import MLPClassifier

# metrics
from sklearn.metrics import accuracy_score, classification_report

# load/save models
from sklearn.externals import joblib

# signed wilcoxon test
from scipy.stats import wilcoxon

In [4]:
# plot
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams.update({'font.size': 22})
fontdict = {'fontsize': 18,
           'weight' : 'bold'}
palette = palette=['green','orange','brown','blue', 'red', 'black', 'yellow']
sns.set()

### load data

In [5]:
# save Scaler
scaler = joblib.load('./data/preprocessamento/zscaler.pkl') 

# save models
X_ros = np.genfromtxt('./data/preprocessamento/X_rox', delimiter=' ')
y_ros = np.genfromtxt('./data/preprocessamento/y_rox', delimiter=' ')

X_smt = np.genfromtxt('./data/preprocessamento/X_smt', delimiter=' ')
y_smt = np.genfromtxt('./data/preprocessamento/y_smt', delimiter=' ')

### test data
X_test = pd.read_csv('./data/preprocessamento/X_test', header=None, sep=' ')
y_test = pd.read_csv('./data/preprocessamento/y_test', header=None, sep=' ')

### Gerando as medias

In [ ]:
means = {'mlp': [], 'rf': []}
X_train_scl, X_test_scl = scaler.transform(X_ros), scaler.transform(X_test)

for i in range(10):
    print("Conjunto: ", i)
    print(means)
    # clfs
    mlp = MLPClassifier(activation='relu', solver='adam', early_stopping=True, 
                        max_iter=200, batch_size=16, validation_fraction=0.1,
                        hidden_layer_sizes=(100))
    rf = RandomForestClassifier(n_estimators=10 , criterion='gini')
    # kfold
    random_state = np.random.randint(2000)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    
    print('RF')
    means['rf'].append(cross_val_score(rf, X_train_scl, y_ros, cv=kfold).mean())
    print('MLP')
    means['mlp'].append(cross_val_score(mlp, X_ros, y_ros, cv=kfold, n_jobs=2).mean())
    print(means)

Conjunto:  0
{'mlp': [], 'rf': []}
RF
MLP
{'mlp': [0.9996481699894757], 'rf': [0.9999790568894322]}
Conjunto:  1
{'mlp': [0.9996481699894757], 'rf': [0.9999790568894322]}
RF


### Wilcoxon Test

In [ ]:
wilcoxon(rf_mean, mlp_mean)

### Acuracia - Boxplot

In [ ]:
sns.boxplot(rf_mean, mlp_mean)